# Vehicle Service Tracker and Cost Estimator

In [1]:
import datetime
import math
import sys
import os
import re
import pdfplumber
import pytesseract
from PIL import Image

# Creating service checklist with intervals and costs in australia 
service_checklist = {
    "Checking the engine oil & oil filter": {"km": 10000, "cost": 150},
    "Inspecting brake pads": {"km": 20000, "cost": 300},
    "Checking coolant flush": {"km": 50000, "cost": 100},
    "Checking transmission fluid": {"km": 80000, "cost": 200},
    "Inspecting timing belt": {"km": 100000, "cost": 500},
    "Tire replacement": {"km": 60000, "cost": 400}
}

FILENAME = "service_history.txt"

def get_user_data():
    try:
        last_service_date = input("Enter the last service date (YYYY-MM-DD): ")
        last_service_date = datetime.datetime.strptime(last_service_date, "%Y-%m-%d")
        present_mileage = int(input("Enter the current mileage (km): "))
        last_service_mileage = int(input("Enter the mileage at last service (km): "))
        avg_km_perday = float(input("Approximately how far does your car travel daily (km): "))
        return last_service_date, present_mileage, last_service_mileage, avg_km_perday
    except ValueError:
        print("Invalid input!")
        sys.exit(1)

def save_service_history(data):
    with open(FILENAME, "a") as file:
        file.write(data + "\n")

def read_service_history():
    if os.path.exists(FILENAME):
        print("\n Last Service History:")
        with open(FILENAME, "r") as file:
            print(file.read())
    else:
        print("\n No previous service records found!")

def wear_and_cost(present_mileage, last_service_mileage, avg_km_perday): #Checking service dues, wear rate and costs
    km_after_last_service = present_mileage - last_service_mileage
    wear_rate = avg_km_perday / 1000  # Assume ware rate as 1000

    print("\n Vehicle Service Status & Cost Estimation")
    estimated_cost = 0
    service_log = f"Service Date: {datetime.datetime.today().strftime('%Y-%m-%d')}\n"

    for service, data in service_checklist.items():
        km = data["km"]
        cost = data["cost"]

        # Calculating wear factor
        wear_factor = math.exp(-km_after_last_service / km)

        if km_after_last_service >= km:
            print(f" {service} is DUE!! Estimated Cost: ${cost}")
            estimated_cost += cost
            service_log += f"{service}: DUE (Cost: ${cost})\n"
        else:
            remaining_km = km - km_after_last_service
            estimated_time = remaining_km / avg_km_perday  # Days until next service
            estimated_date = datetime.datetime.today() + datetime.timedelta(days=estimated_time)
            print(f"{service} is OK. Next service in {remaining_km} km (~{estimated_time:.1f} days, due on {estimated_date.strftime('%Y-%m-%d')}). Wear Factor: {wear_factor:.2f}")
            service_log += f"{service}: Next due on {estimated_date.strftime('%Y-%m-%d')} ({remaining_km} km left)\n"

    print(f"\n Total Service Cost will be ${estimated_cost}")

    if estimated_cost > 1000:
        print(" Your service cost is high!!")

    # Save data to file
    save_service_history(service_log)

def extract_text_from_pdf(pdf_path): ####Extract data from the last service report using pdfplumber  
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None
    return text.strip()

def analyze_service_report(text): #Data analyzing for upcoming services
    text = text.lower()
    suggested_services = []


    service_keywords = {
        "oil": "Checking the engine oil & oil filter",
        "brake": "Inspecting brake pads",
        "coolant": "Checking coolant flush",
        "transmission": "Checking transmission fluid",
        "belt": "Inspecting timing belt",
        "tire": "Tire replacement"
    }

   
    for keyword, service in service_keywords.items():  #Finding manteinance mentioned in the file
        if re.search(rf"\b{keyword}\b", text):
            suggested_services.append(service)

    if suggested_services:
        print("\n We recommend you to check following based on your last service report")
        for service in suggested_services:
            print(f"{service}")
    else:
        print("\n No any additional services found from the report.")

def upload_service_report():
    pdf_path = input("\n Enter the file path to your service report (PDF): ")

    if not os.path.exists(pdf_path):
        print(" Error! File not found. Please enter a valid path.")
        return

    extracted_text = extract_text_from_pdf(pdf_path)

    if extracted_text:
        print("\n Extracted Service Report Content:\n")
        print(extracted_text)
        analyze_service_report(extracted_text)

def main():
    print(" Vehicle Service Checklist & Cost Calculator")
    read_service_history()  # Reading last service records

    choice = input("\n Do you want to Enter data manually, press (1) or Upload a service report PDF press (2)")
    
    if choice == "1":
        last_service_date, present_mileage, last_service_mileage, avg_km_perday = get_user_data()
        wear_and_cost(present_mileage, last_service_mileage, avg_km_perday)
    elif choice == "2":
        upload_service_report()
    else:
        print(" Invalid number! Please a valid number.")
        sys.exit(1)

# Run the program
if __name__ == "__main__":
    main()

 Vehicle Service Checklist & Cost Calculator

 Last Service History:
Service Date: 2025-03-28
Checking the engine oil & oil filter: DUE (Cost: $150)
Inspecting brake pads: Next due on 2026-08-10 (5000 km left)
Checking coolant flush: Next due on 2034-10-27 (35000 km left)
Checking transmission fluid: Next due on 2043-01-13 (65000 km left)
Inspecting timing belt: Next due on 2048-07-05 (85000 km left)
Tire replacement: Next due on 2037-07-23 (45000 km left)



 Do you want to Enter data manually, press (1) or Upload a service report PDF press (2)2

 Enter the file path to your service report (PDF): C:\Users\User\OneDrive - Swinburne University\Standard-Inspection-Report-Sample-pages-deleted.pdf

 Extracted Service Report Content:

Cosmetic Panel / paint = Blue
Minor Scratches / Dents = Yellow
Damaged / Dented = Black
Previous Body Repairs = Red
Rust = Orange
Please Note:
This inspection will not make comment on the alignment of any parts or body panels and will
not be able to detect if 